## **1. Package Installations**

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

In [ ]:
pip install pydantic>=1.10.13

In [ ]:
pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## **2. Imports**

In [ ]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

##**3. Universal NER Model**

In [ ]:
model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

<ipython-input-14-9410ce7de563>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})


### **Prompt template**

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


<ipython-input-15-0d8e09dc8e3e>:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [ ]:
from datasets import load_dataset
dataset = load_dataset("ncbi/ncbi_disease")

README.md:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

ncbi_disease.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

The repository for ncbi/ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ncbi/ncbi_disease.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
# Accessing the data
train_data = dataset['train']
dev_data = dataset['validation']
test_data = dataset['test']

In [ ]:
# test evaluation
text3 = ' '.join(train_data[0]['tokens'])
entity_type="Disease"
print(llm_chain.run({"input_text":text3,"entity_type":entity_type}))
result = llm_chain.run({"input_text":text3,"entity_type":entity_type})
result = result.split('ASSISTANT')[-1].strip()
print(result)


              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
              ASSISTANT: I’ve read this text.
              USER: What describes Disease in the text?
              ASSISTANT:
            ["adenomatous polyposis coli tumour"]
:
            ["adenomatous polyposis coli tumour"]


### **Helper Methods**

In [3]:
def map_entities_to_labels(tokens, entities):
    labels = [0] * len(tokens)  # Initialize all labels to 0 (non-entity)
    for entity in entities:
        start = tokens.index(entity.split()[0])
        end = start + len(entity.split())
        labels[start] = 1  # B-Disease
        for i in range(start + 1, end):
            labels[i] = 2  # I-Disease
    return labels

In [4]:
map_entities_to_labels(['i', 'caught', 'cold', 'and', 'fever'], ['cold and fever'])

[0, 0, 1, 2, 2]

### **Extract Disease entities**

In [ ]:
entity_type="Disease"
n = len(test_data)
disease_tokens_pred = []
pred_converted_to_ncbi = []

for i in range(n):
  print(i, sep=' ')
  text = ' '.join(test_data[i]['tokens'])
  result = llm_chain.run({"input_text":text,"entity_type":entity_type})
  result = result.split('ASSISTANT')[-1].strip()
  disease_tokens_pred.append(result)

In [ ]:
import ast

predicted_ner_tags = []

for i in range(len(test_data)):
  tokens = test_data[i]['tokens']
  cur_disease_pred = disease_tokens_pred[i]
  print(cur_disease_pred)
  print("sent tok: ", tokens)
  list_part = cur_disease_pred[cur_disease_pred.index("["):]

  try:
    converted_list = ast.literal_eval(list_part)
    print(converted_list)
  except (ValueError, SyntaxError):
    print("Input string is not a valid list representation.")

  print(converted_list)

  numeric_labels = map_entities_to_labels(tokens, converted_list)
  predicted_ner_tags.append(numeric_labels)

In [ ]:
final_ls = []
for i in range(len(test_data)):
  cur_disease_pred = disease_tokens_pred[i]
  list_part = cur_disease_pred[cur_disease_pred.index("["):]

  try:
    converted_list = ast.literal_eval(list_part)
  except (ValueError, SyntaxError):
    print("Input string is not a valid list representation.")
  final_ls.append(converted_list)

In [ ]:
predicted_ner_tags_ls = final_ls

### **Add predicted labels for tokens to original DF and save to a file**

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(test_data)

universal_ner_df = df.copy()

universal_ner_df['universal_ner_tags_pred'] = predicted_ner_tags_ls

In [ ]:
universal_ner_df.to_csv('UniversalNER_Predictions.csv', index=False)